# Identifying Prison Conditions, by State

### From the UCLA COVID-19 Behind Bars dataset, identifying and cleaning the policies prisons have put in place to align with social distance recommendations, and support prisoner mental health as a result of the isolation. 
### From the same dataset, identifying the pre-COVID-19 incarcerated population. 
### From the Bureau of Justice Statistics, identifying prison capacity (as of 2018) and incarcerated population (as of 2018). 

In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import re
import distance_policy_wrangle as dpw

### First, extracting the prison population of each state prior to any COVID-19 releases.

In [7]:
population = dpw.import_clean_data('../data/ucla_0519_COVID19_related_prison_releases.csv')

population.head(5)

FileNotFoundError: [Errno 2] File ../data/ucla_0519_COVID19_related_prison_releases.csv does not exist: '../data/ucla_0519_COVID19_related_prison_releases.csv'

### Second, extracting any social distance policies in place in prisons, as well as mitigation policies to attempt to alleviate the effects of isolation on prisoners.

In [5]:
policies = dpw.import_clean_data('../data/may_19/ucla_0519_visitation_policy_by_state.csv')

policies.head(5)

,state,suspended_visitations,explicitly_allows_lawyer_access,compensatory_remote_access_(phone),compensatory_remote_access_(video),effective_date,length,source(s),"additional_notes_(related_activity_suspensions,_explanation_of_compensatory_access,_waivers,_etc.)"
0,Alabama,X,NaN,x,NaN,2020-03-13,30 days,https://www.waaytv.com/content/coronavirus-con...,"also suspended volunteer entry, medical co-pay..."
1,Alaska,X,NaN,X,NaN,2020-03-13,NaN,https://doc.alaska.gov/covid-19,2 free 15-minute calls/week (effective 3/19/20...
2,Arizona,X,NaN,X,NaN,2020-03-13,30 days,https://corrections.az.gov/sites/default/files...,2 x 15 min. calls/wk in addition to normal pho...
3,Arkansas,X,NaN,X,X,2020-03-16,21 days,https://adc.arkansas.gov/images/uploads/COVID_...,Price of phone calls reduced; no connect fee o...
4,California,X,X,X,NaN,2020-03-14,NaN,https://www.cdcr.ca.gov/covid19/,"Beginning March 27, staff and visitors enteri..."


In [4]:
preset_dummies = ["suspended_visitations", "explicitly_allows_lawyer_access", "compensatory_remote_access_(phone)", 
                "compensatory_remote_access_(video)"]
new_cols = ["no_visits", "lawyer_access", "phone_access", "video_access"]

policies = dpw.transform_easy_cols(policies, preset_dummies, new_cols)

In [5]:
#Show the wide variety of policies in place.

for val in policies["additional_notes_(related_activity_suspensions,_explanation_of_compensatory_access,_waivers,_etc.)"].unique():
    print(val)

also suspended volunteer entry, medical co-pays; 3/18: announced compensatory free 15 min phone call once per week 
2 free 15-minute calls/week (effective 3/19/2020). 
2 x 15 min. calls/wk in addition to normal phone call policies/written letter policies. However, all legal and non-legal visitation is suspended (as of 3/18)
Price of phone calls reduced; no connect fee on telephone calls: 15 cents per minute (Div. of Correction & Div. of Community Correction facilities). Video visitation: $2.50 for 30-min visit (state prisons); 15 cents/min for video visits at community correction centers. Rates take effect 03/20/20 (until further notice). Marshall Project: "Legal visits may be granted on a case-by-case basis."
 Beginning March 27, staff and visitors entering CDCR state prisons and community correctional facilities will undergo an additional touchless temperature screening before entering the facility in addition to the ongoing verbal symptom screening put in place on March 14th. Any no

In [6]:
policies = dpw.encode_policies_str(policies, 
                      "additional_notes_(related_activity_suspensions,_explanation_of_compensatory_access,_waivers,_etc.)")

In [7]:
distance_policies = dpw.select_columns(policies)

In [8]:
distance_policies.head(5)

,state,effective_date,no_visits,lawyer_access,phone_access,video_access,no_volunteers,limiting_movement,screening,healthcare_support
0,Alabama,2020-03-13,1,0,1,0,1,0,0,1
1,Alaska,2020-03-13,1,0,1,0,0,0,0,0
2,Arizona,2020-03-13,1,0,1,0,0,0,0,0
3,Arkansas,2020-03-16,1,0,1,1,0,0,0,0
4,California,2020-03-14,1,1,1,0,0,1,1,0


### The final dataframe above shows the policies in place in a given state, and the effective date of those policies, based on the UCLA dataset. 